In [1]:
import numpy as np
import keras
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df_train=pd.read_csv('train.csv')
df_features=df_train.iloc[:,1:785]
df_labels=df_train.iloc[:,0]
df_features=df_features.as_matrix()
df_labels=df_labels.as_matrix()
x_train=df_features[:33600,:]
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_val=df_features[33600:,:]
x_val=x_val.reshape(x_val.shape[0],28,28,1)
y_train = keras.utils.to_categorical(df_labels[:33600], 10)
y_val = keras.utils.to_categorical(df_labels[33600:], 10)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [3]:
input_shape=(28,28,1)
epochs=1
batch_size=128
model = Sequential()
input_shape=(28,28,1)
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [5]:
x_train=x_train.astype('float32'); x_val=x_val.astype('float32');
x_train=x_train/255;x_val=x_val/255;
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


In [6]:
model.fit(x_train, y_train,
          batch_size=batch_size,
         epochs=20,
          verbose=1,
          validation_data=(x_val, y_val))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 106s 3ms/step - loss: 0.3544 - acc: 0.8919 - val_loss: 0.0984 - val_acc: 0.9675
Epoch 2/20
33600/33600 [==============================] - 100s 3ms/step - loss: 0.1125 - acc: 0.9664 - val_loss: 0.0568 - val_acc: 0.9808
Epoch 3/20
33600/33600 [==============================] - 101s 3ms/step - loss: 0.0831 - acc: 0.9749 - val_loss: 0.0500 - val_acc: 0.9844
Epoch 4/20
33600/33600 [==============================] - 100s 3ms/step - loss: 0.0683 - acc: 0.9790 - val_loss: 0.0427 - val_acc: 0.9861
Epoch 5/20
33600/33600 [==============================] - 100s 3ms/step - loss: 0.0569 - acc: 0.9822 - val_loss: 0.0460 - val_acc: 0.9860
Epoch 6/20
33600/33600 [==============================] - 104s 3ms/step - loss: 0.0484 - acc: 0.9855 - val_loss: 0.0419 - val_acc: 0.9875
Epoch 7/20
33600/33600 [==============================] - 102s 3ms/step - loss: 0.0428 - acc: 0.9868 - val_loss: 0.0368 - v

In [7]:
model.save('Best-MNIST-CNN-model.h5');

In [8]:
df_test=pd.read_csv('test.csv');
df_test=df_test.iloc[:,0:784]
x_test=df_test.as_matrix()
x_test=x_test.reshape(x_test.shape[0],28,28,1)
x_test=x_test.astype('float32');
x_test=x_test/255;

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
res=model.predict(x_test)

In [10]:
predict=np.argmax(res,axis=1);

In [11]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predict)+1)), "Label": predict})
submissions.to_csv("best-submis.csv", index=False, header=True)